**Ensemble Learning**
- **CNN**.
- **RNN** implemented using a *LSTM* and *GRU*.

Full repository avaiable on: https://github.com/FrancescoChemello/Deep-Learning-Project/

In [ ]:
"""
Main libraries
"""

import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import tensorflow as tf

import random
import gc
import sys
import os

In [ ]:
"""
For SMOTE & Upsample
"""
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample

In [ ]:
"""
For the class weight
"""
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
"""
For the ensamble learning
"""
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
"""
Only for the metrics analysis
"""
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

The **Onehot Encoding** that transform a *DNA sequence* to a vector of zeros and ones.

In [ ]:
"""
Functions for the onehot encoding
"""

def onehot_encoder(dataset):
    """
    Function that encodes a DNA dataset into a onehot encoding dataset.
    """
    onehot_dataset = [dna_onehot_encoder(dna_string) for dna_string in dataset]
    onehot_dataset_numpy = np.array(onehot_dataset)

    return onehot_dataset_numpy


def dna_onehot_encoder(dna_sequence):
    """
    Function that encodes a single DNA string into a onehot encoding string.
    """
    onehot_dict = {
        'A' : [1, 0, 0, 0],
        'C' : [0, 1, 0, 0],
        'G' : [0, 0, 1, 0],
        'T' : [0, 0, 0, 1]
    }
    encoder = [onehot_dict[nuc] for nuc in dna_sequence]

    return encoder

In [ ]:
def sequence_to_numeric(sequence):
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    return [mapping[nuc] for nuc in sequence]

The **CNN** model: direct from the file *CNN.ipynb* avaiable in the GitHub repository.

In [ ]:
"""
!- CNN Model
"""

class ConvNet(nn.Module):
    # We can use a differnet pool for each layer
    def __init__(self):
        super(ConvNet, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv1d(300, 200, kernel_size=2, padding=1),
            nn.BatchNorm1d(200),
            nn.PReLU(),
            nn.AvgPool1d(2),
            nn.Dropout1d(0.45)
        )

        # I remove random connection to help the convergency
        self.drop_out = nn.Dropout()

        self.layer2 = nn.Sequential(
            nn.Conv1d(200, 100, kernel_size=2, padding=1),
            nn.BatchNorm1d(100),
            nn.PReLU(),
            nn.AvgPool1d(2),
            nn.Dropout1d(0.45)
        )

        # I remove random connection to help the convergency
        self.drop_out_1 = nn.Dropout()

        self.layer3 = nn.Sequential(
            nn.Conv1d(100, 75, kernel_size=2, padding=1),
            nn.BatchNorm1d(75),
            nn.PReLU(),
            nn.MaxPool1d(2),
            nn.Dropout1d(0.3)
        )

        self.layer4 = nn.Sequential(
            nn.Conv1d(75, 50, kernel_size = 2, padding = 1),
            nn.BatchNorm1d(50),
            nn.PReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.3)
        )


        self.layer5 = nn.Sequential(
            nn.Conv1d(50, 32, kernel_size=2, padding=1),
            nn.BatchNorm1d(32),
            nn.PReLU(),
            nn.MaxPool1d(2),
            nn.Dropout1d(0.3)
        )

        # I remove random connection to help the convergency
        self.drop_out_2 = nn.Dropout()

        self.linear1 = nn.Linear(32, 128)
        self.linear2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x=self.drop_out_1(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x=self.drop_out_2(x)
        # Flatten the output for the linear layer
        x = x.view(x.size(0), -1)
        x = self.linear1(x)
        x = self.linear2(x)
        return x

The **RNN** model: direct from the file *LSTM.ipynb* avaiable in the GitHub repository.

In [ ]:
class LSTM_Net (nn.Module):
    def __init__ (self):
        super(LSTM_Net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)

        self.lstm = nn.LSTM(input_size=128, hidden_size=128, num_layers=6, batch_first=True, dropout=0.5, bidirectional=True)
        # self.lstm = nn.LSTM(input_size=4, hidden_size=128, num_layers=2, dropout=0.5, batch_first=True)

        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)

        x = x[:, -1, :]
        x = x.contiguous().view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

The **RNN** model: direct from the file *GRU.ipynb* avaiable in the GitHub repository.

In [ ]:
class GRU_Net (nn.Module):
    def __init__(self):
        super(GRU_Net, self).__init__()
        self.conv1 = nn.Conv1d(300, 16, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(2)
        self.gru = nn.GRU(16, 64, num_layers=2, bidirectional=True)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = x.permute(0, 2, 1)
        x, _ = self.gru(x)
        x = x.flatten(1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [ ]:
"""
Class to create a Data Loader with X label and Y label together
"""
class CreateDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_point = self.data[idx]
        label = self.labels[idx]
        return data_point, label

**MAIN**

Set **Hyperparameters** for the code:


*   Select the path for the dataset:
    1.   For local machine.
    2.   For Colab enviroment.
    3.   For Goole Drive.

*   Number of data upsampled.
*   Number of data generated by the SMOTE.
*   Type of RNN:
    1.   For the GRU.
    2.   For the LSTM





In [ ]:
"""
Hyperparameters
"""

# Dataset path
dataset_path = 3

# Number of upsampled
num_upsampled = 40000

# Number of SMOTE generated data
num_smote = 80000

# Type of RNN
type_RNN = 1

Training phase

Extraction of the data from the *.cvc* files, **oversample** and **SMOTE** of data from class 1 *(if the user select them)*.

In [ ]:
"""
!- MAIN
"""

# Set the device to be used (GPU or CPU)
device = th.device("cuda" if th.cuda.is_available() else "cpu")
print("Device: ", device)

# Read the input from the cvc file
train_data_path = ""
val_data_path = ""
test_data_path = ""

if dataset_path == 1:
    """
    If runs in local machine:
    """

    print("Local path selected")

    data_dir = os.path.abspath(os.path.join(os.getcwd(), 'data'))
    train_data_path = os.path.join(data_dir, 'fullset_train.csv')
    val_data_path = os.path.join(data_dir, 'fullset_validation.csv')
    test_data_path = os.path.join(data_dir, 'fullset_test.csv')
elif dataset_path == 2:

      print("Colab path selected")

      rel_path_train = '/content/fullset_train.csv'
      rel_path_val = '/content/fullset_validation.csv'
      rel_path_test = '/content/fullset_test.csv'
elif dataset_path == 3:

      print("Google Drive path selected")

      from google.colab import drive
      drive.mount('/content/drive')

      rel_path_train = '/content/drive/MyDrive/Colab Notebooks/fullset_train.csv'
      rel_path_val = '/content/drive/MyDrive/Colab Notebooks/fullset_validation.csv'
      rel_path_test = '/content/drive/MyDrive/Colab Notebooks/fullset_test.csv'
else:
      print("Not a valid path selected!")
      sys.exit(1)

# Training Set

# Read the input from the csv file
train_csv = pd.read_csv(rel_path_train, sep=",")
# Drop the NaN values
train_csv = train_csv.dropna()
# Describe the data
print(train_csv.describe())

# Get the data from the csv file
train_data = train_csv.values
# m = number of input samples
m = train_data.shape[0]

# Dataframe and upsample
data = {'sequence' : train_data[:m,1],
        'label' : train_data[:m,2].astype(np.int32) }

df = pd.DataFrame(data)

count_class_0, count_class_1 = df['label'].value_counts()

print("Samples class 0: ", count_class_0)
print("Samples class 1: ", count_class_1)

df_class_0 = df[df['label'] == 0]
df_class_1 = df[df['label'] == 1]


# Upsample
df_minority_upsampled = resample(df_class_1, replace=True, n_samples=num_upsampled, random_state=42)

df_upsampled = pd.concat([df_class_0, df_minority_upsampled])

count_class_0, count_class_1 = df_upsampled['label'].value_counts()

print("Samples class 0 AFTER upsample: ", count_class_0)
print("Samples class 1 AFTER upsample: ", count_class_1)


# Smote
X_train = df_upsampled['sequence'].values
Y_train = df_upsampled['label'].values

#Convert the sequences to numeric
numeric_train = [sequence_to_numeric(seq) for seq in X_train]
numeric_train = np.array(numeric_train)

# Reshape the data
n_samples, _ = numeric_train.shape
numeric_train_sequences = numeric_train.reshape((n_samples, -1))

# Apply the SMOTE algorithm:
smote = SMOTE(sampling_strategy={1: num_smote}, random_state=42)
X_train, Y_train = smote.fit_resample(numeric_train_sequences, Y_train)

#Convert the sequences to string
X_train = [''.join([['A', 'C', 'G', 'T'][nuc] for nuc in seq]) for seq in X_train]

data_SM = {'sequence' : X_train,
           'label' : Y_train }

df_SM = pd.DataFrame(data_SM)

count_class_0, count_class_1 = df_SM['label'].value_counts()

print("Samples class 0 AFTER upsample and SMOTE: ", count_class_0)
print("Samples class 1 AFTER upsample and SMOTE: ", count_class_1)


# OneHot encoding for the training data
print("Start onehot encoding for the training data")
X_train = onehot_encoder(X_train)

# Convert the data to a tensor
X_train = th.from_numpy(X_train).to(device)
Y_train = th.tensor(Y_train).to(device)

print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)

# Free memory
# del train_csv, train_data, m, numeric_train, numeric_train_sequences, smote, data, df
del train_csv, train_data, m, data, df
gc.collect()
th.cuda.empty_cache()

# Validation Set

# Read the input from the csv file
val_csv = pd.read_csv(rel_path_val, sep=",")
# Drop the NaN values
val_csv = val_csv.dropna()
# Describe the data
print(val_csv.describe())

val_data = val_csv.values
# m = number of input samples
m = val_data.shape[0]

X_val = val_data[:m,1]
Y_val = val_data[:m,2].astype(np.int32)

# OneHot encoding for the validation data
print("Start onehot encoding for the validation data")
X_val = onehot_encoder(X_val)

X_val = th.from_numpy(X_val).to(device)
Y_val = th.tensor(Y_val).to(device)

print("X_val shape", X_val.shape)
print("Y_val shape", Y_val.shape)

# Free memory
del val_csv, val_data, m
gc.collect()
th.cuda.empty_cache()

# Test

# Read the input from the csv file
test_csv = pd.read_csv(rel_path_test, sep=",")
# Drop the NaN values
test_csv = test_csv.dropna()
# Describe the data
print(test_csv.describe())

test_data = test_csv.values
# m = number of input samples
m = test_data.shape[0]

X_test = test_data[:m,1]
Y_test = test_data[:m,2].astype(np.int32)

# OneHot encoding for the test data
print("Start onehot encoding for the test data")
X_test = onehot_encoder(X_test)

X_test = th.from_numpy(X_test).to(device)
Y_test = th.tensor(Y_test).to(device)

print("X_test shape", X_test.shape)
print("Y_test shape", Y_test.shape)

# Free memory
del test_csv, test_data, m
gc.collect()
th.cuda.empty_cache()

**Weight class** and **Data Loader**

In [ ]:
"""
Function to calculate the weight for the two classes for
"""
def get_class_weights(dataset):
    labels = [label for label in dataset]
    class_counts = np.bincount(labels)
    total_samples = len(labels)
    class_weights = total_samples / (len(class_counts) * class_counts)
    return th.tensor(class_weights, dtype=th.float)

# Calculate the class weights for the X_train
class_weights = get_class_weights(Y_train.cpu()).to(device)

In [ ]:
# Create the Dataset
train_dataset = CreateDataset(X_train, Y_train)
val_dataset = CreateDataset(X_val, Y_val)
test_dataset = CreateDataset(X_test, Y_test)

# Batch size
batch_dim = 128

# Create the Data Loader
train_loader = DataLoader(train_dataset, batch_size=batch_dim, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_dim, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_dim, shuffle=True)

# Free memory
del X_train, X_val, Y_val
gc.collect()
th.cuda.empty_cache()

*Train* of the **CNN** using the *Validation* set to check the performance. We use *stop early* to prevent overfitting.

In [ ]:
print("Start training the CNN")

model_CNN = ConvNet().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = th.optim.AdamW(model_CNN.parameters(), lr=0.001)
scheduler = th.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)

# Early stopping parameters
patience = 5  # CHECK THE VALUE!!
best_val_loss = float('inf')
counter = 0

# Training the model
num_epochs = 100  # Hope to reach convergence before 100 epoches

for epoch in range(num_epochs):
    model_CNN.train()
    running_loss = 0.0
    i = 0
    for X_batch, Y_batch in train_loader:
        X_batch = X_batch.float()
        Y_batch = Y_batch.long()

        # Forward pass
        outputs = model_CNN(X_batch.to(device))
        loss = criterion(outputs, Y_batch.to(device))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        i += 1

        # Free memory
        del X_batch, Y_batch
        th.cuda.empty_cache()

    # Validation the model
    model_CNN.eval()
    val_loss = 0.0
    j = 0
    with th.no_grad():
        for X_batch, Y_batch in val_loader:
            X_batch = X_batch.float()
            Y_batch = Y_batch.long()

            outputs = model_CNN(X_batch.to(device))
            loss = criterion(outputs, Y_batch.to(device))

            val_loss += loss.item()
            j += 1

            # Free memory
            del X_batch, Y_batch
            th.cuda.empty_cache()

    # Losses
    running_loss = running_loss/i
    val_loss = val_loss/j

    scheduler.step(val_loss)

    # Check for early stopping
    if val_loss <= best_val_loss:
        best_val_loss = val_loss
        counter = 0
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}, Patience: {patience-counter}')
    else:
        counter += 1
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}, Patience: {patience-counter}')
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

print("End training the CNN")

# Free memory
del i, j, running_loss, val_loss
gc.collect()
th.cuda.empty_cache()

**(OPTIONAL):** *Test* results for the **CNN**.

In [ ]:
print("Start testing the CNN")

# Model ready for the evaluation
model_CNN.eval()

# Arrays to save the results
all_preds = []
all_labels = []

# Testing the model
with th.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch = X_batch.to(device).float()
        Y_batch = Y_batch.to(device).long()

        outputs = model_CNN(X_batch)
        _, predicted = th.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(Y_batch.cpu().numpy())

        # Free memory
        del X_batch, Y_batch
        th.cuda.empty_cache()

# Convert the tensor to use scikit learn metrics
y_true_CNN = all_labels
y_pred_CNN = all_preds

# Metrics
accuracy_CNN = accuracy_score(y_true_CNN, y_pred_CNN)
precision_CNN = precision_score(y_true_CNN, y_pred_CNN, average='weighted', zero_division=0)
recall_CNN = recall_score(y_true_CNN, y_pred_CNN, average='weighted')
f1_CNN = f1_score(y_true_CNN, y_pred_CNN, average='weighted')
auroc_CNN = roc_auc_score(y_true_CNN, y_pred_CNN)

print(f'Accuracy: {accuracy_CNN:.6f}')
print(f'Precision: {precision_CNN:.6f}')
print(f'Recall: {recall_CNN:.6f}')
print(f'F1-score: {f1_CNN:.6f}')
print(f'AUROC: {auroc_CNN:.6f}')

print("End testing the CNN")

# Free memory
del all_labels, all_preds, y_true_CNN, y_pred_CNN, accuracy_CNN, precision_CNN, recall_CNN, f1_CNN, auroc_CNN
gc.collect()
th.cuda.empty_cache()

In [ ]:
"""
Select the RNN.
"""
if type_RNN == 1:
    print("GRU net selected!")
    model_RNN = GRU_Net().to(device)
elif type_RNN == 2:
    print("LSTML net selected!")
    model_RNN = LSTM_Net().to(device)
else:
    print("Not a valid net selected!")
    sys.exit(1)

*Train* of the **RNN** using the *Validation* set to check the performance. We use *stop early* to prevent overfitting.

In [ ]:
print("Start training the RNN")

# Model, loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = th.optim.Adam(model_RNN.parameters(), lr=0.001)

# Early stopping parameters
patience = 5  # CHECK THE VALUE!!
best_val_loss = float('inf')
counter = 0

# Training the model
num_epochs = 100  # Hope to reach convergence before 100 epoches

for epoch in range(num_epochs):
    model_RNN.train()
    running_loss = 0.0
    i = 0
    for X_batch, Y_batch in train_loader:

        X_batch = X_batch.to(device).float()
        Y_batch = Y_batch.to(device).long()

        # Forward pass
        outputs = model_RNN(X_batch.to(device))
        loss = criterion(outputs, Y_batch.to(device))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        i += 1

        # Free memory
        del X_batch, Y_batch
        th.cuda.empty_cache()

    # Validation
    model_RNN.eval()
    val_loss = 0.0
    j = 0
    with th.no_grad():
        for X_batch, Y_batch in val_loader:
            X_batch = X_batch.to(device).float()
            Y_batch = Y_batch.to(device).long()

            outputs = model_RNN(X_batch)
            loss = criterion(outputs, Y_batch)

            val_loss += loss.item()
            j += 1

            # Free memory
            del X_batch, Y_batch
            th.cuda.empty_cache()

    # Losses
    running_loss = running_loss/i
    val_loss = val_loss/j

    # Check for early stopping
    if val_loss <= best_val_loss:
        best_val_loss = val_loss
        counter = 0
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}, Patience: {patience-counter}')
    else:
        counter += 1
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}, Patience: {patience-counter}')
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

print("End training the RNN")

# Free memory
del i, j, running_loss, val_loss
gc.collect()
th.cuda.empty_cache()

**(OPTIONAL):** *Test* results for the **RNN**.

In [ ]:
print("Start testing the RNN")

# Model ready for the evaluation
model_RNN.eval()

# Arrays to save the results
all_preds = []
all_labels = []

# Testing the model
with th.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch = X_batch.to(device).float()
        Y_batch = Y_batch.to(device).long()

        outputs = model_RNN(X_batch)
        _, predicted = th.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(Y_batch.cpu().numpy())

# Convert the tensor to use scikit learn metrics
y_true_RNN = all_labels
y_pred_RNN = all_preds

# Metrics
accuracy_RNN = accuracy_score(y_true_RNN, y_pred_RNN)
precision_RNN = precision_score(y_true_RNN, y_pred_RNN, average='weighted', zero_division=0)
recall_RNN = recall_score(y_true_RNN, y_pred_RNN, average='weighted')
f1_RNN = f1_score(y_true_RNN, y_pred_RNN, average='weighted')
auroc_RNN = roc_auc_score(y_true_RNN, y_pred_RNN)

print(f'Accuracy: {accuracy_RNN:.6f}')
print(f'Precision: {precision_RNN:.6f}')
print(f'Recall: {recall_RNN:.6f}')
print(f'F1-score: {f1_RNN:.6f}')
print(f'AUROC: {auroc_RNN:.6f}')

print("End testing the RNN")

# Free memory
del all_labels, all_preds, y_true_RNN, y_pred_RNN, accuracy_RNN, precision_RNN, recall_RNN, f1_RNN, auroc_RNN
gc.collect()
th.cuda.empty_cache()

Implementation of the **Ensemble Learning** using the *stacking ensemble learing* paradigm.

In [ ]:
"""
Function that, given a model and a data loader, it returns all the predictions (outputs)
"""
def get_predictions(model_pred, data_loader):
    model_pred.eval()
    all_preds = []
    with th.no_grad():
        for inputs, _ in data_loader:
            outputs = model_pred(inputs.float().to(device))
            all_preds.append(outputs.cpu().numpy())
    return np.concatenate(all_preds)

# Prepare the prediction for the training model
CNN_train_preds = get_predictions(model_CNN, train_loader)
LSTM_train_preds = get_predictions(model_RNN, train_loader)

# Prepare the prediction for the testing model
CNN_test_preds = get_predictions(model_CNN, test_loader)
LSTM_test_preds = get_predictions(model_RNN, test_loader)

# Concatenate the two inputs
X_train_meta = np.hstack((CNN_train_preds, LSTM_train_preds))
X_test_meta = np.hstack((CNN_test_preds, LSTM_test_preds))

print("Shape of X_train_meta: ", X_train_meta.shape)
print("Shape of Y_train: ", Y_train.shape)

print("Shape of X_test_meta: ", X_test_meta.shape)
print("Shape of Y_test: ", Y_test.shape)

# Define the train loader
train_dataset = CreateDataset(X_train_meta, Y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_dim, shuffle=True)

# Free memory
gc.collect()
th.cuda.empty_cache()

**Test** of the performance.

In [ ]:
# We use a polynomial SVM for the final classification
model_SVM = SVC(kernel = 'poly', probability = True)

# Train the model
for X_batch, Y_batch in train_loader:
    X_batch = X_batch.cpu()
    Y_batch = Y_batch.cpu()
    model_SVM.fit(X_batch, Y_batch.ravel())

In [ ]:
print("Start testing the final model - SVM")

# I convert the data (if needed)
y_true = Y_test.cpu().numpy()

# Testing the meta-model: SVM
y_pred = model_SVM.predict(X_test_meta)
y_pred_prob = model_SVM.predict_proba(X_test_meta)[:, 1]

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
auroc = roc_auc_score(y_true, y_pred_prob)

print(f'Accuracy: {accuracy:.6f}')
print(f'Precision: {precision:.6f}')
print(f'Recall: {recall:.6f}')
print(f'F1-score: {f1:.6f}')
print(f'AUROC: {auroc:.6f}')

print("End testing the final model - SVM")

# Free memory
del y_pred, y_pred_prob, y_true, accuracy, precision, recall, f1, auroc
gc.collect()
th.cuda.empty_cache()

In [ ]:
model_XGB = xgb.XGBClassifier()

# Train the model
for X_batch, Y_batch in train_loader:
    X_batch = X_batch.cpu()
    Y_batch = Y_batch.cpu()
    model_XGB.fit(X_batch, Y_batch.ravel())

In [ ]:
print("Start testing the final model - XGB")

# I convert the data (if needed)
y_true = Y_test.cpu().numpy()

# Testing the meta-model: XGB
y_pred = model_XGB.predict(X_test_meta)
y_pred_prob = model_XGB.predict_proba(X_test_meta)[:, 1]

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
auroc = roc_auc_score(y_true, y_pred_prob)

print(f'Accuracy: {accuracy:.6f}')
print(f'Precision: {precision:.6f}')
print(f'Recall: {recall:.6f}')
print(f'F1-score: {f1:.6f}')
print(f'AUROC: {auroc:.6f}')

print("End testing the final model - XGB")

# Free memory
del y_pred, y_pred_prob, y_true, accuracy, precision, recall, f1, auroc
gc.collect()
th.cuda.empty_cache()

In [ ]:
model_RF = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
for X_batch, Y_batch in train_loader:
    X_batch = X_batch.cpu()
    Y_batch = Y_batch.cpu()
    model_RF.fit(X_batch, Y_batch.ravel())

In [ ]:
print("Start testing the final model - RF")

# I convert the data (if needed)
y_true = Y_test.cpu().numpy()

# Testing the meta-model: RF
y_pred = model_RF.predict(X_test_meta)
y_pred_prob = model_RF.predict_proba(X_test_meta)[:, 1]

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
auroc = roc_auc_score(y_true, y_pred_prob)

print(f'Accuracy: {accuracy:.6f}')
print(f'Precision: {precision:.6f}')
print(f'Recall: {recall:.6f}')
print(f'F1-score: {f1:.6f}')
print(f'AUROC: {auroc:.6f}')

print("End testing the final model - RF")

# Free memory
del y_pred, y_pred_prob, y_true, accuracy, precision, recall, f1, auroc
gc.collect()
th.cuda.empty_cache()

In [ ]:
model_GBC = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# Train the model
for X_batch, Y_batch in train_loader:
    X_batch = X_batch.cpu()
    Y_batch = Y_batch.cpu()
    model_GBC.fit(X_batch, Y_batch.ravel())

In [ ]:
print("Start testing the final model - GBC")

# I convert the data (if needed)
y_true = Y_test.cpu().numpy()

# Testing the meta-model: GBC
y_pred = model_GBC.predict(X_test_meta)
y_pred_prob = model_GBC.predict_proba(X_test_meta)[:, 1]

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
auroc = roc_auc_score(y_true, y_pred_prob)

print(f'Accuracy: {accuracy:.6f}')
print(f'Precision: {precision:.6f}')
print(f'Recall: {recall:.6f}')
print(f'F1-score: {f1:.6f}')
print(f'AUROC: {auroc:.6f}')

print("End testing the final model - GBC")

# Free memory
del y_pred, y_pred_prob, y_true, accuracy, precision, recall, f1, auroc
gc.collect()
th.cuda.empty_cache()